In [1]:
import tensorflow as tf

In [8]:
filenames = "gs://mom_seguros_images_car/ic/train/train/*/*.png"

In [9]:
filepath = tf.io.gfile.glob(filenames)

In [16]:
filepath

['gs://mom_seguros_images_car/ic/train/train/correct/0.9526_car_b7cdd35a-68a9-11ed-b6d3-ce59abab15a2.png',
 'gs://mom_seguros_images_car/ic/train/train/correct/0.9541_car_d1d8232c-68d6-11ed-b6d3-ce59abab15a2.png',
 'gs://mom_seguros_images_car/ic/train/train/correct/0.9548_car_a5fcb34e-68db-11ed-b6d3-ce59abab15a2.png',
 'gs://mom_seguros_images_car/ic/train/train/correct/0.9549_car_4cab15a4-68e2-11ed-b6d3-ce59abab15a2.png',
 'gs://mom_seguros_images_car/ic/train/train/correct/0.9549_car_7475b666-68d3-11ed-b6d3-ce59abab15a2.png',
 'gs://mom_seguros_images_car/ic/train/train/correct/0.9560_car_04a6da4e-68de-11ed-b6d3-ce59abab15a2.png',
 'gs://mom_seguros_images_car/ic/train/train/correct/0.9561_car_2c814fae-68b6-11ed-b6d3-ce59abab15a2.png',
 'gs://mom_seguros_images_car/ic/train/train/correct/0.9561_car_343d7ab6-68d8-11ed-b6d3-ce59abab15a2.png',
 'gs://mom_seguros_images_car/ic/train/train/correct/0.9565_car_a9cf3d18-68c5-11ed-b6d3-ce59abab15a2.png',
 'gs://mom_seguros_images_car/ic/trai